# OMERO Micro-SAM Workflow

In [ ]:
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline,
)

from omero_annotate_ai.core.config import load_config_from_yaml
import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")

try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Install with: pip install -e .[omero]")

try:
    import keyring
    print("Keyring support: Available")
except ImportError:
    print("Keyring support: Not available")

## OMERO Connection

In [ ]:
conn_widget = create_omero_connection_widget()
conn_widget.display()

In [ ]:
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established")

print(f"Connected as: {conn.getUser().getName()}")
print(f"Group: {conn.getGroupFromContext().getName()}")

In [ ]:
# Create the enhanced configuration widget with table management
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

## Pipeline Execution

In [ ]:
config = load_config_from_yaml(
    "C:\\Users\\Maarten\\AppData\Local\Temp\\omero_annotations_m_14q2cq\\annotation_config.yaml"
)
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.training.trainingset_name}")
print(f"Model: {config.micro_sam.model_type}")
print(f"Output: {config.batch_processing.output_folder}")

if config.patches.use_patches:
    print(f"Patches: {config.patches.patches_per_image} per image ({config.patches.patch_size[0]}×{config.patches.patch_size[1]})")

print(f"Scope: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")

In [ ]:
pipeline = create_pipeline(config, conn)

container = conn.getObject(config.omero.container_type.capitalize(), config.omero.container_id)
if container is None:
    raise ValueError(f"{config.omero.container_type} with ID {config.omero.container_id} not found")

print(f"Container: {container.getName()}")

images_list = pipeline.get_images_from_container()
print(f"Images to process: {len(images_list)}")

for i, img in enumerate(images_list[:3]):
    print(f"  {i+1}. {img.getName()} (ID: {img.getId()})")
if len(images_list) > 3:
    print(f"  ... and {len(images_list) - 3} more")

In [ ]:
# Step 1: Create annotation table
table_id, images_list = pipeline.create_annotation_table(images_list)
print(f"Table created with ID: {table_id}")




In [ ]:
# Step 2: Run annotation processing
table_id, processed_images = pipeline.run_annotation(table_id, images_list)

print(f"Completed. Processed {len(processed_images)} images")
print(f"Table ID: {table_id}")

## Results

In [ ]:
print(f"Training Set: {config.training.trainingset_name}")
print(f"Table ID: {table_id}")
print(f"Images Processed: {len(processed_images)}")
print(f"Output: {config.batch_processing.output_folder}")

if processed_images:
    print(f"Processed Images:")
    for i, img in enumerate(processed_images[:5]):
        if hasattr(img, 'getName'):
            print(f"  {i+1}. {img.getName()} (ID: {img.getId()})")
        else:
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"  {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(processed_images) > 5:
        print(f"  ... and {len(processed_images) - 5} more")

In [ ]:
config_filename = f"annotation_config_{config.training.trainingset_name}.yaml"
config_path = Path(config.batch_processing.output_folder) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Config saved: {config_path}")
except Exception as e:
    config.save_yaml(config_filename)
    print(f"Config saved: {config_filename}")

## Cleanup

In [ ]:
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed")

print(f"Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"Table ID: {table_id if 'table_id' in locals() else 'N/A'}")

## Usage Options

```python
# Option 1: Full workflow
table_id, processed_images = pipeline.run_full_workflow()

# Option 2: Split workflow
table_id, images_list = pipeline.create_annotation_table()
table_id, processed_images = pipeline.run_annotation(table_id, images_list)

# Option 3: Resume from existing table
table_id, processed_images = pipeline.run_annotation(existing_table_id)
```